In [1]:
import os

In [2]:
import numpy as np
from scipy.spatial import KDTree

# here is cells to read files

## chage to directory of dataset

In [3]:
data_path = "../data/pointclouds"
data_files = os.listdir(data_path)

## iterate each sample files and label it

000 - 099: building
100 - 199: car
200 - 299: fence
300 - 399: pole
400 - 499: tree

In [4]:
points_cluster_label=[]
points_cluster_kdtree = []
def read_data(cluster_label,point_cluster,file,data_path):
    file_path = data_path+"/"+file
    #print(file_path)
    #print(os.getcwd())
    
    with open(file_path,'r',encoding='utf-8') as f:
        line_points = f.readlines()
        inserted_point = []
        
        file_code = file.split(".")[0]
        file_code = int(file_code)
        # label it according to the index of files
        file_label = file_code//100
        for one_line in line_points:
            x,y,z = [float(coord) for coord in one_line.strip().split(" ")]
            #print(file,one_line)
            inserted_point.append([x,y,z])
        #print(inserted_point)
        # store data structure is kdtree
        one_tree = KDTree(np.array(inserted_point))
        
        cluster_label.append(file_label)
        point_cluster.append(one_tree)
for file in data_files:
    
    read_data(points_cluster_label,points_cluster_kdtree,file,data_path)
    
print(len(points_cluster_label))
print(len(points_cluster_kdtree))

500
500


In [5]:
multi_scales = [10,20,30,40,50]
points_label=[]
points_features=[]
points_eigenvalues=[]
cluster_eigenvalues=[]
sample_labels=[]

In [6]:
def extract_cluster_eigenvalues(points_cluster_kdtree,points_cluster_label,\
                               sample_labels,cluster_eigenvalues):
    for OneTree,label in zip(points_cluster_kdtree,points_cluster_label):
        selected_points = OneTree.data
        
            
        cov_martrix = np.cov(selected_points,rowvar=False)
            
        eigen_values = np.linalg.eigvals(cov_martrix)
        eigen_values.sort()
        eigen_values= list(eigen_values)
        lambda1,lambda2,lambda3 = eigen_values
        #eigen_values.append((lambda3-lambda2)/lambda3)
        #eigen_values.append((lambda2-lambda1)/lambda3)
        #eigen_values.append((lambda1)/lambda3)
        Z_RANGE = max(selected_points[:,2])-min(selected_points[:,2])
        Y_RANGE =max(selected_points[:,1])-min(selected_points[:,1])
        X_RANGE =max(selected_points[:,0])-min(selected_points[:,0])
        eigen_values+=list(cov_martrix[0])
        eigen_values.append(Z_RANGE)
        eigen_values.append(X_RANGE)
        eigen_values.append(Y_RANGE)
        
        
        cluster_eigenvalues.append(eigen_values)
            
        sample_labels.append(label)
extract_cluster_eigenvalues(points_cluster_kdtree,points_cluster_label,\
                               sample_labels,cluster_eigenvalues)
import pickle
with open("../data"+"/"+"cluster_eigenvalues.txt",'wb') as sf:
    pickle.dump(cluster_eigenvalues, sf, pickle.HIGHEST_PROTOCOL)  

with open("../data"+"/"+"cluster_labels.txt",'wb') as sf:
    pickle.dump(sample_labels, sf, pickle.HIGHEST_PROTOCOL)

In [7]:
len(sample_labels)

500

In [8]:
def extract_points_eigenvalues(points_cluster_kdtree,points_cluster_label,\
                               points_label,multi_scale):
    scale = max(multi_scales)
    for OneTree,label in zip(points_cluster_kdtree,points_cluster_label):
        points = OneTree.data
        for point in points:
            #print(point)
            d,selected_points =OneTree.query(x=point, k=scale, workers=5)
            selected_points = points[selected_points]
            
            cov_martrix = np.cov(selected_points,rowvar=False)
            
            eigen_values = np.linalg.eigvals(cov_martrix)
            points_eigenvalues.append(eigen_values)
            
            points_label.append(label)

extract_points_eigenvalues(points_cluster_kdtree,points_cluster_label,\
                               points_label,multi_scales)       

In [9]:
import pickle
with open("../data"+"/"+"eigenvalues.txt",'wb') as sf:
    pickle.dump(points_eigenvalues, sf, pickle.HIGHEST_PROTOCOL)

In [10]:
print(len(points_label))

1276507


In [11]:
print(len(points_eigenvalues))
print(file)

1276507
034.xyz


In [12]:

with open("../data"+"/"+"point_labels.txt",'wb') as sf:
    pickle.dump(points_label, sf, pickle.HIGHEST_PROTOCOL)